<h1>ME309: Structural Mechanics</h1> <div style="text-align: right"><h4>Pietro Zullo</h4> </div>                                                          
<h3>Final Exam 2020</h3>           <div style="text-align: right"> <h4>U41494036</h4> </div>                                                           

Consider a cantilever beam that is supported by a roller at its opposite end. The beam is loaded at its center by a force P. Element (1) has a bending rigidity that is four times higher than element (2), and take L = 1.2m, E = 210GPa, I = 1.5 × 107mm4, and P = 30kN.
First, use the appropriate energy method to determine an approximate shape of the deformed shape, and plot the shape using Mathematica or MATLAB. Second, implement the beam element in your FEA model, and use it to determine the nodal displacements and rotations. Compare the two solutions and discuss the results.
Please print and attach your FEA code.

In [1]:
import numpy as np 
import sympy as sys 
import matplotlib.pyplot as plt

In [2]:

from IPython.display import Math, HTML

def load_mathjax_in_cell_output():
  display(HTML("<script src='https://www.gstatic.com/external_hosted/"
               "mathjax/latest/MathJax.js?config=default'></script>"))
get_ipython().events.register('pre_run_cell', load_mathjax_in_cell_output)

sys.init_printing() 

In [3]:
y,L = sys.symbols('y L')
global a0,a1,a2,a3,a4
a0, a1, a2, a3, a4 = sys.symbols('a0 a1 a2 a3 a4') # deflection w(x) = a1+a2*x+a3*x^2+a4*x^3
w1, w2, theta_1, theta_2 = sys.symbols('w1 w2 theta_1 theta_2') # degrees of freedom (DoF)
E, I = sys.symbols('E I') # materials and geometry
F1, M1, F2, M2 = sys.symbols('F1 M1 F2 M2') # forces (y direction) and couples
P, q = sys.symbols('P q') # force, force/length

We will first assume that the deflection takes the form of a polynomial function of 4th grade 

In [4]:
def w(y):
    return a0+a1*y+a2*y**2+a3*y**3+a4*y**4

In [5]:
w(y)

Now we have to decide what boundary conditions to include 

<h3> Geometric Boundary Conditions </h3>

I will impose three Dirichlet Boundary conditions ddefined by the joints of the beam, the clamp in zero sets both the displacement and the rotation at y = 0 equal to zero, the roller in y = L sets the vertical displacement w(L) equal to zero. 

In [6]:
gbc = [sys.Eq(0,w(0)),
sys.Eq(0,w(2*L)),
      sys.Eq(0,sys.diff(w(y),y).subs(y,0))]

               

In [7]:
sol = sys.solve([gbc[0],gbc[1],gbc[2]],(a0,a2,a1))

In [8]:
sol

In [9]:
a0 = sol[a0]
a1 = sol[a1]
a2 = sol[a2]

In [10]:
w(y)

In [11]:
sys.diff(w(y),y,2)

Definition of the strain energy 

In [12]:
U = (E*I/2)*sys.integrate(sys.diff(w(y),y,2)**2,(y,0,L))+(E*I/8)*sys.integrate(sys.diff(w(y),y,2)**2,(y,L,2*L))

In [13]:
U


In [14]:
sys.simplify(U)

Definition of the work done by point load P

In [15]:
W = - P * w(L)
sys.simplify(W)


Definition of the total potential energy of the beam 

In [16]:
PI = U+W

Rayleigh-Reiz Method, we set the derivatives of the total potential energy in respect to the unknown constants equal to zero and solve for the constants 

In [17]:
sol.update(sys.solve([sys.Eq(0,sys.diff(PI,a3)),sys.Eq(0,sys.diff(PI,a4))],(a3,a4)))

In [18]:
sol


In [19]:
a3 = sol[a3]
a4 = sol[a4]

So we have all the values of the constants 

In [20]:
a0

In [21]:
a1

In [22]:
a2


In [23]:
a3

In [24]:
a4

We have to insert the values of a3 and a4 into a2 